In [1]:
import copy

import pymdp
from pymdp import utils
from pymdp.agent import Agent
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [33]:
trials = 32
steps = 2  # number of timepoints per trial

#STATES
# state if the img is cs+ or cs-
state = ['cs+', 'cs-']


#OBSERVATIONS

# morphing level binary. shock only if morph_level=6
morph_level_obs = ['0','1']
# the subject receive the electric shock?
shock_obs = ['shock', 'no shock','null']
# observation to discriminate if the subject is surprised or prepared. A sort of diff between user prediction and shock observation
surprised_obs = ['not_surprised','surprised','null']

#ACTIONS
# the subject predict if should be receive the shock or not
action = ['guess cs+','guess cs-']

# Define `num_states` and `num_factors` below
num_states = [len(state)] #[2]
num_factors = len(num_states) #1

#Define `num_obs` and `num_modalities` below
num_obs = [len(morph_level_obs), len(shock_obs), len(surprised_obs)] # [2,2,2]
num_modalities = len(num_obs) #3



In [34]:
len(action)

2

In [35]:
B_ = np.zeros((num_states[0], num_states[0], len(action)))
B_

array([[[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]]])

In [36]:
def matrices(TrialEnv):
    """
    The A, B, C, D, E matrices are built.
    """
    # This is the likelihood mapping that truly describes the relationship between the
    # environment's hidden state and the observations the agent will get

    # A matrix
    # A dimension?
    A = utils.obj_array(num_modalities)
    A_morph = np.zeros((len(morph_level_obs), len(state)))
    A_shock = np.zeros((len(shock_obs), len(state)))
    A_surprise = np.zeros((len(surprised_obs), len(state)))
    #mor_lev_binary/states
    A_morph = np.array([ [0, 1],
                         [1, 0] ]
                          )
    #shock-noshock/states
    A_shock = np.array([ [0.75, 0],
                         [0.25, 1],
                         [0, 0]]
                          )
    #prepared-surprised/states
    A_surprise = np.array([ [0.5, 0.5],
                            [0.5, 0.5],
                            [0, 0]]
                          )
    A[0] = A_morph
    A[1] = A_shock
    A[2] = A_surprise
    # B matrix
    # The transition mapping that truly describes the environment's dynamics

    #B as an identity matrix. no state transitions
    B = utils.obj_array(num_factors)
    B_ = np.zeros((num_states[0], num_states[0], len(action)))

    B_[:,:,0] = np.eye(num_states[0])
    B_[:,:,1] = np.eye(num_states[0])

    '''B_[0, :,:] = np.array([ [1, 0],
                           [0, 1] ])
    B_[1, :,:] = np.array([ [1, 0],
                       [0, 1] ])'''

    B[0]=B_
    B_gm = copy.deepcopy(B)  # make a copy of the true transition likelihood to initialize the transition model
    #???
    #A_gm = copy.deepcopy(A_gp)  # make a copy of the true observation likelihood to initialize the observation model
    #???
    #B_gm = copy.deepcopy(B_gp)  # make a copy of the true transition likelihood to initialize the transition model


    # This is the matrix representing the preferences over outcomes
    #prepared-surprised/cs+-
    C = utils.obj_array_zeros(num_obs)          #num modalities
    C[0] = np.array([4.0,-6.0])
    C[1] = np.zeros(3)
    C[2] = np.zeros(3)


    return A, B_gm, C

In [37]:
class TrialEnv(object):
    def __init__(self,data):
        """
        Constructor of the environment
        :param data: Dataframe with 4 columns: 'trial_number','morphing_level','condition','shock'

        'trial_number': number of trial in the experiment
        'morphing_level': 1 if cs+, 0 otherwise
        'condition': 1 cs+, 0 otherwise
        'shock': 1 if the shock is given at this trial, 0 otherwise

        """
        self.data = data

    def get_observation(self,param,trial_number):
        '''

        :param param: column of the dataframe that you want to read
        :param trial_number:
        :return: value of the specific column in the trial_number row
        '''
        value= str(self.data.iloc[int(trial_number)][param])

        if param == 'shock':
            if value == '0':
                value = 'shock'
            if value == '1':
                value = 'no shock'

        return value

    def step(self, action,current_trial):
        '''
        Take a step in the environment given an action

        :param action:
        :param current_trial:
        :return:
        '''
        surprised_obs = None

        if action == 'guess cs+':
            shock_obs = self.get_observation(param='shock',trial_number=current_trial)

            if shock_obs == 'shock': # in this case shock and i have predicted cs+ #TODO change this
                surprised_obs = self.get_surprised_low()

            else: # in this case no shock and i have predicted cs+
                surprised_obs = self.get_surprised_high()


        if action == 'guess cs-':
            shock_obs = self.get_observation(param='shock',trial_number=current_trial)

            if shock_obs == 'shock': # in this case shock and prediction cs- #TODO change this
                surprised_obs = self.get_surprised_high()

            else: # in this case no shock and prediction cs-
                surprised_obs = self.get_surprised_low()

        if surprised_obs is None:
            print('ERROR: Action not supported!')

        return surprised_obs


    def get_surprised_high(self):
        return 'surprised'

    def get_surprised_low(self):
        return 'not_surprised'


In [38]:
A,B,C = matrices(TrialEnv)
B[0]

array([[[1., 1.],
        [0., 0.]],

       [[0., 0.],
        [1., 1.]]])

In [39]:
def load_data(columns=[], binary_morphing_level=False, shock_as_int = False,subject_number=0,remove_first_trials=None,remove_last_trials=None):
    '''
    Function that read NewLookAtMe files and extract
    :param columns:
    :param binary_morphing_level:
    :param shock_as_int:
    :param subject_number:
    :return:
    '''

    df = pd.read_csv('./data/newLookAtMe/newLookAtMe01.csv')
    df = df.filter(columns)

    if 'morphing level' in columns and binary_morphing_level:
        df['morphing level'] = [int(d==6) for d in df['morphing level']]
    if 'shock' in columns and shock_as_int:
        df['shock'] = df['shock'].astype(int)
    if remove_first_trials is not None:
        df = df[remove_first_trials:]
        df.reset_index(drop=True,inplace=True)
    return df

In [40]:
data= load_data(columns=['morphing level','shock'] ,binary_morphing_level=True, shock_as_int = True,remove_first_trials=16)
data

,morphing level,shock
0,1,1
1,0,0
2,0,0
3,1,1
4,0,0
...,...,...
139,0,0
140,0,0
141,0,0
142,0,0


In [41]:
# define an agent
my_agent = Agent(A = A, B = B, C = C)

# define an environment
my_env = TrialEnv(data= data)


initial_action = 'first_action'

# iterate over all trials
for trial in range (trials) :
    print("\n-----TRIAL "+str(trial)+ " ------")

    for step in range (steps) : #each trial divided in two steps

        agent_stimul_obs = my_env.get_observation(param='morphing level',trial_number=trial)

        if step == 0:
            agent_surpr_obs = 'null'
            agent_shock_obs = 'null'

        if step == 1:
            agent_shock_obs = my_env.get_observation(param='shock',trial_number=trial)
            print('observation:',agent_shock_obs)

        obs_ = [morph_level_obs.index(agent_stimul_obs), shock_obs.index(agent_shock_obs), surprised_obs.index(agent_surpr_obs)]
        state_distr_inferred = my_agent.infer_states (observation=obs_) # agent update beliefs about hidden states given observations

        my_agent.infer_policies () # inferring policies and sampling actions from the posterior

        agent_action = my_agent.sample_action()
        action_name = action[int(agent_action)]
        if step==0:
            agent_surpr_obs = my_env.step(action_name,trial)
            print('observation is cs+: '+agent_stimul_obs)
            print('distribution states:',state_distr_inferred)
            print('action done: '+action_name)



-----TRIAL 0 ------
observation is cs+: 1
distribution states: [array([0.5, 0.5])]
action done: guess cs+
observation: no shock

-----TRIAL 1 ------
observation is cs+: 0
distribution states: [array([1.e+00, 9.e-16])]
action done: guess cs+
observation: shock

-----TRIAL 2 ------
observation is cs+: 0
distribution states: [array([1.0e+00, 1.1e-15])]
action done: guess cs+
observation: shock

-----TRIAL 3 ------
observation is cs+: 1
distribution states: [array([1.0e+00, 1.3e-15])]
action done: guess cs+
observation: no shock

-----TRIAL 4 ------
observation is cs+: 0
distribution states: [array([1.e+00, 1.e-16])]
action done: guess cs+
observation: shock

-----TRIAL 5 ------
observation is cs+: 1
distribution states: [array([1.e+00, 3.e-16])]
action done: guess cs+
observation: no shock

-----TRIAL 6 ------
observation is cs+: 1
distribution states: [array([1.e+00, 1.e-16])]
action done: guess cs+
observation: no shock

-----TRIAL 7 ------
observation is cs+: 0
distribution states: [a